In [21]:
import torch
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import sys

In [22]:
def get_train_test(classes):
  transform = transforms.Compose(
      [transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

  SEED = 1

  # CUDA?
  cuda = torch.cuda.is_available()
  print("CUDA Available?", cuda)

  # For reproducibility
  torch.manual_seed(SEED)

  if cuda:
    torch.cuda.manual_seed(SEED)

  # dataloader arguments - something you'll fetch these from cmdprmt
  dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

  trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                      download=True, transform=transform)

  testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transform)

  trainloader = torch.utils.data.DataLoader(trainset, **dataloader_args)
  testloader = torch.utils.data.DataLoader(testset, **dataloader_args)

  return trainloader, testloader

In [30]:
import torch.nn as nn
import torch.nn.functional as F
import sys

#Model C1C2C3C40 (basically 3 MPs)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.dropout_value = 0.1
       
        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        ) #RF: 3

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        ) #RF: 5

        # TRANSITION BLOCK 1
        self.pool1 = nn.MaxPool2d(2, 2) #RF: 6

        # Depthwise Separable Convolution
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3), padding=1, groups=32, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        ) #RF: 10

        # CONVOLUTION BLOCK 2
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        ) #RF: 14

        # TRANSITION BLOCK 2
        self.pool2 = nn.MaxPool2d(2, 2) #RF: 16

        # Dilated Convolution
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1, dilation=2, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        ) #RF: 24

        # TRANSITION BLOCK 3
        self.pool3 = nn.MaxPool2d(1,1) #RF:24

        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        ) #32

        # OUTPUT BLOCK
        self.gap = nn.AdaptiveAvgPool2d(output_size=1) #RF: 40
        self.convblock7 = nn.Conv2d(in_channels=128, out_channels=10, kernel_size=(1, 1), padding=0, bias=False) #RF: 56


    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.pool1(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.pool2(x)
        x = self.convblock5(x)
        x = self.pool3(x)
        x = self.convblock6(x)
        x = self.gap(x)
        x = self.convblock7(x)
        x = x.view(-1, 10)
        
        return F.log_softmax(x, dim=-1)  



In [24]:

def train(model, device, train_loader, optimizer, epoch):
  train_losses = []
  train_acc = []

  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()

    # Predict
    y_pred = model(data)

    #loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)
    
  return train_losses,train_acc

In [25]:
def test(model, device, test_loader):
  try:
    model.eval()
    test_loss = 0
    correct = 0
    test_losses=[]
    test_acc = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_losses.append(test_loss)
    test_acc.append(100. * correct / len(test_loader.dataset))
  except Exception as e:
    print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno) + " " + type(e).__name__ + " " + str(e))
    sys.exit(1)
  return test_losses,test_acc

In [26]:
def plot_acc_loss(test_losses, test_acc):
  try:
  
    fig, (ax1, ax2) = plt.subplots(2, figsize=(15,10))
    fig.suptitle('Test Loss and Test Accuracy for the models', fontsize=16)
    ax1.plot(test_losses[0])

    ax1.set_title("Test Loss")
    
    ax2.plot(test_acc[0])

    ax2.set_title("Test Accuracy")
   
    plt.savefig('acc_vs_loss.jpg')
    plt.show()
  except Exception as e:
        print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno) + " " + type(e).__name__ + " " + str(e))
        sys.exit(1)

In [27]:
def main(classes):
  !pip install torchsummary
  from torchsummary import summary
  from tqdm import tqdm
  
  train_loader, test_loader = get_train_test(classes)

  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  print(device)

  model = Net().to(device)
  summary(model, input_size=(3, 32, 32))
  optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
  EPOCHS=25
 
  test_loss=[]
  test_accuracy=[]
  for epoch in range(EPOCHS):
      print("EPOCH:", epoch + 1)
      train_losses,train_acc=train(model, device, train_loader, optimizer, epoch=EPOCHS)
      test_losses,test_acc=test(model, device, test_loader)

In [31]:
if __name__ == '__main__':
  from tqdm import tqdm
  if hasattr(tqdm, '_instances'):
    tqdm._instances.clear()

  classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog','frog', 'horse', 'ship', 'truck')
  
  main(classes)

CUDA Available? True
Files already downloaded and verified
Files already downloaded and verified
cuda


  0%|          | 0/391 [00:00<?, ?it/s]

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             864
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
           Dropout-4           [-1, 32, 32, 32]               0
            Conv2d-5           [-1, 32, 32, 32]           9,216
       BatchNorm2d-6           [-1, 32, 32, 32]              64
              ReLU-7           [-1, 32, 32, 32]               0
           Dropout-8           [-1, 32, 32, 32]               0
         MaxPool2d-9           [-1, 32, 16, 16]               0
           Conv2d-10           [-1, 32, 16, 16]             288
      BatchNorm2d-11           [-1, 32, 16, 16]              64
             ReLU-12           [-1, 32, 16, 16]               0
          Dropout-13           [-1, 32, 16, 16]               0
           Conv2d-14           [-1, 64,

Loss=1.1965550184249878 Batch_id=390 Accuracy=49.07: 100%|██████████| 391/391 [00:15<00:00, 25.65it/s]
  0%|          | 0/391 [00:00<?, ?it/s]


Test set: Average loss: 1.2476, Accuracy: 5515/10000 (55.15%)

EPOCH: 2


...


Test set: Average loss: 0.4917, Accuracy: 8316/10000 (83.16%)

EPOCH: 24


Loss=0.38121575117111206 Batch_id=390 Accuracy=86.54: 100%|██████████| 391/391 [00:15<00:00, 25.66it/s]
  0%|          | 0/391 [00:00<?, ?it/s]


Test set: Average loss: 0.5076, Accuracy: 8275/10000 (82.75%)

EPOCH: 25


Loss=0.4506608545780182 Batch_id=390 Accuracy=86.58: 100%|██████████| 391/391 [00:14<00:00, 26.11it/s]



Test set: Average loss: 0.4984, Accuracy: 8265/10000 (82.65%)

